# Reading input data and building the SFS

Now that our models are set up, we are ready to read out input data and convert it into site frequency spectra.

In [1]:
from delimitpy import parse_input
from delimitpy import process_empirical
import numpy as np
import os
import pickle

# Parse configuration files and read in intermediate files from previous part of tutorial.

We need to read our config file into memory again, and we need to read the files that were created in the 'Building a Model Set' notebook.

The test data we are using was simulated under a model with three populations and no gene flow. 

In [2]:
# read the configuraiton file
config_parser = parse_input.ModelConfigParser("../../examples/test1/config.txt")
config_values = config_parser.parse_config()

# read the labels and parameterized files
labels = np.load(os.path.join(config_values["output directory"], 'labels.npy'), allow_pickle=True)
with open(os.path.join(config_values["output directory"], 'parameterized_models.pickle'), 'rb') as f:
    parameterized_models = pickle.load(f)


# Read empirical data, and convert it into a numpy array.

First, generate our data processor. Then, we convert our folder of fasta files into a numpy array. We will keep the same number of sites that we kept for simulated data. Missing data will be encoded as -1 (any sites in the alignment other than A, T, C, and G will be converted to -1).

In [3]:
# create our data processor, and convert our fastas to a numpy array
data_processor = process_empirical.DataProcessor(parameterized_models, config=config_values)
empirical_array = data_processor.fasta_to_numpy()
print(empirical_array.shape) # print the shape of our array: (individuals, SNPs).

(30, 1038)


# Choose projection for site-frequency-spectrum

SFS cannot be generated from datasets that include missing data. To circumvent this, we use a downsampling approach such as that described in Satler and Carstens (2017, Molecular Ecology, doi: 10.1111/mec.14137.) We must choose thresholds for each populations (i.e., the minumum number of individuals that must be sampled for a SNP to be used.) To help with this, we use the function find_downsampling from the class DataProcessor. This function generates a dictionary that holds the number of SNPs that meet each threshold.

In [4]:
# generate dictionary with the number of SNPs at different sampling thresholds
empirical_2d_sfs_sampling = data_processor.find_downsampling(empirical_array)

# print threshold, SNP combos for thresholds with at least 1030 snps
minspns = 1030
min1000 = {key: value for key, value in empirical_2d_sfs_sampling.items() if value >= minspns}
print(min1000)


{(1, 1, 1): 1038, (1, 1, 2): 1038, (1, 1, 3): 1038, (1, 1, 4): 1038, (1, 1, 5): 1038, (1, 1, 6): 1038, (1, 1, 7): 1037, (1, 2, 1): 1038, (1, 2, 2): 1038, (1, 2, 3): 1038, (1, 2, 4): 1038, (1, 2, 5): 1038, (1, 2, 6): 1038, (1, 2, 7): 1037, (1, 3, 1): 1038, (1, 3, 2): 1038, (1, 3, 3): 1038, (1, 3, 4): 1038, (1, 3, 5): 1038, (1, 3, 6): 1038, (1, 3, 7): 1037, (1, 4, 1): 1038, (1, 4, 2): 1038, (1, 4, 3): 1038, (1, 4, 4): 1038, (1, 4, 5): 1038, (1, 4, 6): 1038, (1, 4, 7): 1037, (1, 5, 1): 1038, (1, 5, 2): 1038, (1, 5, 3): 1038, (1, 5, 4): 1038, (1, 5, 5): 1038, (1, 5, 6): 1038, (1, 5, 7): 1037, (1, 6, 1): 1035, (1, 6, 2): 1035, (1, 6, 3): 1035, (1, 6, 4): 1035, (1, 6, 5): 1035, (1, 6, 6): 1035, (1, 6, 7): 1034, (2, 1, 1): 1038, (2, 1, 2): 1038, (2, 1, 3): 1038, (2, 1, 4): 1038, (2, 1, 5): 1038, (2, 1, 6): 1038, (2, 1, 7): 1037, (2, 2, 1): 1038, (2, 2, 2): 1038, (2, 2, 3): 1038, (2, 2, 4): 1038, (2, 2, 5): 1038, (2, 2, 6): 1038, (2, 2, 7): 1037, (2, 3, 1): 1038, (2, 3, 2): 1038, (2, 3, 3): 10

# Down-project the SFS

Now, we must choose a threshold. I will choose (9,6,7), as it is the largest combination of thresholds that gives us more than 1030 SNPs.

In [5]:
# build 10 replicates of the 2d SFS
empirical_2d_sfs = data_processor.numpy_to_2d_sfs(empirical_array, downsampling={"A":9, "B":7, "C":6}, replicates = 10)

# build 10 replicates of the mSFS
empirical_msfs = data_processor.numpy_to_msfs(empirical_array, downsampling={"A":9, "B": 7, "C":6}, replicates = 10)